In [1]:
from datetime import datetime
import time
import pickle
import glob

import matplotlib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

from sklearn.preprocessing import LabelEncoder
from sklearn.cross_validation import StratifiedShuffleSplit
from skimage.io import imread
from skimage.transform import resize
from tensorflow.python.framework.ops import reset_default_graph

from util import etl, helpers

/usr/local/lib/python2.7/dist-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
# loading data and setting up constants
TRAIN_PATH = "./data/train.csv"
TEST_PATH = "./data/test.csv"
IMAGE_PATHS = glob.glob("./data/images/*.jpg")
IMAGE_SHAPE = (128, 128, 1)
HEIGHT, WIDTH, CHANNEL = IMAGE_SHAPE

with open('./data/train.pickle', 'rb') as f:
    train = pickle.load(f)
with open('./data/test.pickle', 'rb') as f:
    test = pickle.load(f)

In [3]:
#Training Loop
BATCH_SIZE = 64
ITERATIONS = 1e4
LOG_FREQ = 10
VALIDATION_SIZE = 0.1 # 0.1 is ~ 100 samples for validation
SEED = 42
DROPOUT = False
LEARNING_RATE = 0.0005
VALID_EVERY = 100
TRAIN_SIZE = 1.0
CLASS_SIZE = 0.1dr

NUM_CLASSES = 99   # len(train['species'].unique())
NUM_FEATURES = 64 # for all three features, margin, shape and texture
dropout = 0.75 # Dropout, probability to keep units

x = tf.placeholder(tf.float32, [BATCH_SIZE, WIDTH, HEIGHT, CHANNEL], name='x_image_pl')
y = tf.placeholder(tf.float32, [BATCH_SIZE, int(round(NUM_CLASSES * CLASS_SIZE))], name='classes_pl')
keep_prob = tf.placeholder(tf.float32) #dropout (keep probability)


In [4]:
int(round(NUM_CLASSES * CLASS_SIZE))

10

In [5]:
# Store layers weight & bias
weights = {
    # 5x5 conv, 1 input, 32 outputs
    'wc1': tf.Variable(tf.random_normal([5, 5, 1, 32]), name='wc1'),
    # 5x5 conv, 32 inputs, 64 outputs
    'wc2': tf.Variable(tf.random_normal([5, 5, 32, 64]), name='wc2'),
    # fully connected, 7*7*64 inputs, 1024 outputs
    'wd1': tf.Variable(tf.random_normal([WIDTH / 4 * HEIGHT / 4 * 64, 1024]), name='wd1'),
    # 1024 inputs, 10 outputs (class prediction)
    'out': tf.Variable(tf.random_normal([1024, int(round(NUM_CLASSES * CLASS_SIZE))]), name='out')
}

biases = {
    'bc1': tf.Variable(tf.random_normal([32]), name='bc1'),
    'bc2': tf.Variable(tf.random_normal([64]), name='bc2'),
    'bd1': tf.Variable(tf.random_normal([1024]), name='bd1'),
    'out': tf.Variable(tf.random_normal([int(round(NUM_CLASSES * CLASS_SIZE))]), name='b_out')
}

# Construct model
net = {}
pred = helpers.conv_net(x, weights, biases, keep_prob, net)

# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(pred, y))
optimizer = tf.train.AdamOptimizer(learning_rate=LEARNING_RATE).minimize(cost)

# Evaluate model
correct_pred = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initializing the variables
init = tf.initialize_all_variables()

(64, 128, 128, 32)
(64, 64, 64, 32)
(64, 64, 64, 64)
(64, 32, 32, 64)
(64, 1024)
(64, 10)
Instructions for updating:
Use `tf.global_variables_initializer` instead.


In [6]:
batch_gen = etl.batch_generator(train, test, batch_size=BATCH_SIZE, num_classes=NUM_CLASSES,
                            num_iterations=ITERATIONS, seed=SEED, train_size=TRAIN_SIZE, val_size=VALIDATION_SIZE, class_size=CLASS_SIZE)

train_batch = batch_gen.gen_train()
valid_batch = batch_gen.gen_valid()
test_batch = batch_gen.gen_test()


valid_batch_one, i = valid_batch.next()

initiating batch generator with (10/99) classes and 1.000000 of the training data and 0.100000 as validation


util/etl.py:24: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  out[row, col] = 1


In [7]:
print len(batch_gen.gen_train().next()['images'])
print len(batch_gen.gen_train().next()['ts'])
print len(valid_batch_one['images'])
print len(valid_batch_one['ts'])

64
64
64
64


In [8]:
print "Iter \t Batch Loss \t Batch Accuracy \t Valid Loss \t Valid Accuracy \t Time delta\n"
report_interval = 1
time_last = time.time()
with tf.Session() as sess:
    sess.run(init)
    # Keep training until reach max iterations
    for i, batch in enumerate(batch_gen.gen_train()):
        sess.run(optimizer, feed_dict={
            x: batch['images'],
            y: batch['ts'],
            keep_prob: dropout
        })
        if i % report_interval == 0:
            # Calculate batch loss and accuracy
            batch_loss, batch_acc = sess.run([cost, accuracy], feed_dict={
                x: batch['images'],
                y: batch['ts'],
                keep_prob: 1.
            })
            valid_loss, valid_acc = sess.run([cost, accuracy], feed_dict={
                x: valid_batch_one['images'],
                y: valid_batch_one['ts'],
                keep_prob: 1.
            })
            print "%d\t%.4f\t\t%.4f\t\t%.4f\t\t%.4f\t\t%.4f\n" % (i, batch_loss, batch_acc, valid_loss, valid_acc, time.time() - time_last)
            time_last = time.time()
        if i >= ITERATIONS:
            break
        if valid_acc > 0.99:
            break
    print "Optimization Finished!"
    
    print "Testing Accuracy:", \
    
    

Iter 	 Batch Loss 	 Batch Accuracy 	 Valid Loss 	 Valid Accuracy 	 Time delta

0	245024.8906		0.0625		39886.5508		0.0000		15.3657

1	179932.7188		0.1719		29430.7871		0.0469		8.2335

2	179376.3750		0.1094		27848.4531		0.0312		7.7480

3	128158.9844		0.2812		25913.7266		0.0312		7.6652

4	142131.6719		0.2969		23154.5664		0.0312		7.8009

5	103789.6875		0.4688		19720.3828		0.0469		7.7339

6	85416.8125		0.4844		16401.5957		0.0469		7.8944

7	50065.6172		0.5625		11585.0723		0.0469		7.6148

8	39951.7539		0.6094		8202.8027		0.0938		7.8520

9	18888.7109		0.7500		10186.3496		0.0781		8.5856

10	42908.8984		0.6406		11550.4346		0.0625		9.7851

11	43476.2227		0.6406		11403.1934		0.0781		8.7765

12	30974.9902		0.7656		10864.5762		0.0781		8.7234

13	32168.0293		0.6562		9346.5361		0.1094		9.1532

14	28907.0352		0.7344		7269.4619		0.1094		10.9239

15	8145.3252		0.8594		4960.6909		0.1094		8.7191

16	5713.1396		0.8906		3253.5322		0.1250		8.5990

17	8391.8828		0.8594		2856.5293		0.1250		8.0507

18	1242.6726		

KeyboardInterrupt: 

In [ ]:
#correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
#accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
loss, accuracy, argmax_y = loss_and_acc(y)

In [ ]:
batch, i = valid_batch.next()
print(accuracy.eval(feed_dict={x: batch['features'], ts_pl: batch['ts']}))